# 1. Symplectic representation

## 1.1. On Humphries generators

### Sp_repr class and functions

In [1]:
%display latex

In [2]:
class Sp_repr:
    #--- 2x2 ---
    I = identity_matrix(2)
    O = zero_matrix(ZZ,2)
    J = matrix([[0,1],[-1,0]])
    #---
    L = matrix([[1,0],[1,1]])
    K = matrix([[0,-1],[0,0]])
    #--- 4x4 ---
    J4 = block_matrix([[J, O], [O, J]])
    # J4 = J4_bm.subs([(J, evJ)]).as_explicit()
    #---
    CHR_2_MTX = {'a': block_matrix([[L.inverse(), O], [O, I]]),
                 'b': block_matrix([[L.T, K], [K, L.T]]),
                 'c': block_matrix([[I, O], [O, L.inverse()]]),
                 'd': block_matrix([[I, O], [O, L.T]]),
                 'f': block_matrix([[L.T, O], [O,I]])}
    #---
    
    def __init__(self, l:str):
        self.loop = l
        self.matrix = self.CHR_2_MTX.get(l) if l.islower() else self.CHR_2_MTX.get(l.lower()).inverse()

In [3]:
def is_Sp(M):
    J = Sp_repr.J4
    return M.transpose()*J*M == J

list(map(lambda l: is_Sp(Sp_repr(l).matrix), ['a','b','c','d','f', 'A'])) 

[True, True, True, True, True, True]

## 1.2. The symplectic images of Abcd and abcD

In [4]:
a,b,c,d,f = tuple(map(lambda l: Sp_repr(l).matrix, ['a','b','c','d','f']))
J4 = Sp_repr.J4

a,b,c,d,f, J4 

(
[ 1  0| 0  0]  [ 1  1| 0 -1]  [ 1  0| 0  0]  [1 0|0 0]  [1 1|0 0]
[-1  1| 0  0]  [ 0  1| 0  0]  [ 0  1| 0  0]  [0 1|0 0]  [0 1|0 0]
[-----+-----]  [-----+-----]  [-----+-----]  [---+---]  [---+---]
[ 0  0| 1  0]  [ 0 -1| 1  1]  [ 0  0| 1  0]  [0 0|1 1]  [0 0|1 0]
[ 0  0| 0  1], [ 0  0| 0  1], [ 0  0|-1  1], [0 0|0 1], [0 0|0 1],

[ 0  1| 0  0]
[-1  0| 0  0]
[-----+-----]
[ 0  0| 0  1]
[ 0  0|-1  0]
)

In [5]:
A, B, C, D, F = (k.inverse() for k in [a,b,c,d,f])
A,B,C,D,F

(
[1 0 0 0]  [ 1 -1  0  1]  [1 0 0 0]  [ 1  0  0  0]  [ 1 -1  0  0]
[1 1 0 0]  [ 0  1  0  0]  [0 1 0 0]  [ 0  1  0  0]  [ 0  1  0  0]
[0 0 1 0]  [ 0  1  1 -1]  [0 0 1 0]  [ 0  0  1 -1]  [ 0  0  1  0]
[0 0 0 1], [ 0  0  0  1], [0 0 1 1], [ 0  0  0  1], [ 0  0  0  1]
)

In [6]:
M = a*b*c*D
display(M)
N = A*b*c*d
display(N)

list(map(is_Sp, [M, N]))

[ 1  1  1 -2]
[-1  0 -1  2]
[ 0 -1  0  1]
[ 0  0 -1  2]

[ 1  1  1  0]
[ 1  2  1  0]
[ 0 -1  0  1]
[ 0  0 -1  0]

[True, True]

### The charactoristic polynomial of M (and N)

In [7]:
R.<a> = PolynomialRing(ZZ)
cp_M = M.charpoly('t'); display(cp_M, cp_M.factor())
cp_N = N.charpoly('t'); #display(cp_N)

print(f"cp_M == cp_N ?   --> {cp_M == cp_N}")

t^4 - 3*t^3 + 3*t^2 - 3*t + 1

t^4 - 3*t^3 + 3*t^2 - 3*t + 1

cp_M == cp_N ?   --> True


# 2. The ideal for fibered knots

In [8]:
K.<a> = NumberField(cp_M)
OK = K.ring_of_integers()
print("整数基:", OK.basis())
print("類数:", OK.class_number())

U = UnitGroup(K)
print(U.gens_values())

G = K.galois_group(); print(G)
# # ガロア群の元を列挙
# for sigma in G:
#     # 共役写像を適用
#     conjugated_a = sigma(a)
#     if conjugated_a == 1/a:
#         print(f"σ = {sigma.tuple()}")
#         print(f"σ(a) = {conjugated_a}")

tau = K.hom(1/a, K); print(f"{tau(a) = }")

整数基: [1, a, a^2, a^3]
類数: 1
[-1, a - 1, -a^3 + 3*a^2 - 3*a + 2]
Galois group 4T3 (D(4)) with order 8 of t^4 - 3*t^3 + 3*t^2 - 3*t + 1
tau(a) = -a^3 + 3*a^2 - 3*a + 3


In [9]:
def ev_wrt_a(X):
    X_K = X.change_ring(K)
    evlist = X_K.eigenvectors_right();
    v = [v[1][0] for v in evlist if v[0] == a][0]
    if X_K*v == a*v:
        return v
    else:
        return None

### Eigenvectors of M and N with respect to the eigen value $a$

In [10]:
v, w = [ev_wrt_a(X) for X in (M,N)]
display(v, w)
print("------- Check -------")
print(M*v == a*v, N*w == a*w)

(1, -a^3 + 2*a^2 - a + 1, -3*a^3 + 8*a^2 - 6*a + 6, -2*a^3 + 5*a^2 - 4*a + 4)

(1, a^3 - 2*a^2 + a - 1, -a^3 + 2*a^2, a^2 - 2*a)

------- Check -------
True True


### Ideals of M and N

In [11]:
# The integral ideal (v) is equal to ZZ[a]

T = matrix([[1,0,0,0],[0,1,0,0],[0,0,-1,1],[0,0,1,-2]]); display(T, T*v)
T2 = matrix([[1,0,0,0],[0,1,0,1],[0,0,1,-1],[0,0,0,1]]); display(T2*(T*v))
T3 = matrix([[1,0,0,0],[1,1,0,0],[0,0,-1,0],[0,-2,-2,1]]); display(T3*(T2*(T*v)))
X = T3*T2*T; display(X)

[ 1  0  0  0]
[ 0  1  0  0]
[ 0  0 -1  1]
[ 0  0  1 -2]

(1, -a^3 + 2*a^2 - a + 1, a^3 - 3*a^2 + 2*a - 2, a^3 - 2*a^2 + 2*a - 2)

(1, a - 1, -a^2, a^3 - 2*a^2 + 2*a - 2)

(1, a, a^2, a^3)

[ 1  0  0  0]
[ 1  1  1 -2]
[ 0  0  2 -3]
[ 0 -2  3 -4]

In [12]:
# The integral ideal (w) is equal to ZZ[a]

T = matrix([[1,0,0,0],[0,1,1,0],[0,0,0,1],[0,0,1,0]]); display(T, T*w)
T2 = matrix([[1,0,0,0],[1,1,0,0],[2,2,1,0],[4,4,2,-1]]); display(T*w, T2*(T*w))
Y = T2*T; display(Y)

[1 0 0 0]
[0 1 1 0]
[0 0 0 1]
[0 0 1 0]

(1, a - 1, a^2 - 2*a, -a^3 + 2*a^2)

(1, a - 1, a^2 - 2*a, -a^3 + 2*a^2)

(1, a, a^2, a^3)

[1 0 0 0]
[1 1 1 0]
[2 2 2 1]
[4 4 3 2]

In [13]:
S = Y.inverse()*X
display(S, S.inverse())

print(f"{(S*v == w) = }")
display(S*v, w)
print(f"{is_Sp(S)=}")

[ 1  0  0  0]
[ 0 -1  0  0]
[ 0  2  1 -2]
[-2 -2  0  1]

[ 1  0  0  0]
[ 0 -1  0  0]
[ 4 -2  1  2]
[ 2 -2  0  1]

(S*v == w) = True


(1, a^3 - 2*a^2 + a - 1, -a^3 + 2*a^2, a^2 - 2*a)

(1, a^3 - 2*a^2 + a - 1, -a^3 + 2*a^2, a^2 - 2*a)

is_Sp(S)=False


### The values of M and N

In [14]:
# 双一次形式を関数として定義
def bilinear_form(x, y, A = identity_matrix(ZZ, 4)):
    """
    行列Aで定義される双一次形式b(x, y) = x^T * A * y を計算する
    """
    return x.dot_product(A * y)

In [15]:
df = diff(cp_M)
Del = df(a)/a; display(Del)

3*a^3 - 5*a^2 - 3

In [16]:
def Va(v):
    tau_v = vector([tau(e) for e in v])
    aD = bilinear_form(v, tau_v, J4)
    return aD/Del

In [17]:
[Va(vec) for vec in [v,w]]

[-a^3 + 3*a^2 - 2*a + 3, a^3 - 3*a^2 + 2*a - 1]

In [18]:
(v, Va(v)), (w, Va(w))

(((1, -a^3 + 2*a^2 - a + 1, -3*a^3 + 8*a^2 - 6*a + 6, -2*a^3 + 5*a^2 - 4*a + 4),
  -a^3 + 3*a^2 - 2*a + 3),
 ((1, a^3 - 2*a^2 + a - 1, -a^3 + 2*a^2, a^2 - 2*a), a^3 - 3*a^2 + 2*a - 1))

In [19]:
c = Va(v)/Va(w)
display(c, c == tau(c))

2*a^3 - 6*a^2 + 4*a - 5

True

In [20]:
# --- 判定処理 ---
def check_decomposition(x, sigma, O):
    """
    x が u*sigma(u) の形に分解できるか検証する
    """
    # x が代数的整数であるか確認
    if x not in O:
        print(f"警告: {x} は代数的整数ではありません。")
        
    print(f"\n--- 元 x = {x} の検証 ---")
    
    # 必要条件: x が共役写像 σ で不変であるかチェック
    if sigma(x) != x:
        print(f"不変性チェック失敗: σ(x) = {sigma(x)} は x と異なります。")
        print("したがって、x は u * σ(u) の形には分解できません。")
        return False
    
    print(f"不変性チェック成功: σ(x) = {sigma(x)} で、x と一致します。")
    print("分解できる可能性があります。")
    
    # # u * σ(u) = x となる u の存在を調べる
    # # この部分の実装は、一般には困難
    # # 幸いにも、このケースでは K^* のコホモロジー群が自明なため、
    # # 不変元は常に u*σ(u) の形に書けます。
    # # よって、不変性チェックが成功すれば、分解可能と判定します。
    # # 実際には、x が代数的整数であるかどうかのチェックも重要です。
    # print("理論上、不変元であるため、単数 u が存在して分解できます。")
    
    # ノルムを計算
    x_norm = x.norm()
    print(f"ノルム: {x_norm}")

    # 平方数であるかチェック
    if is_square(x_norm):
        print("ノルムは平方数です。分解できる可能性があります。")
    else:
        print("ノルムは平方数ではありません。分解できません。")
        
    return True

# 検証を実行
check_decomposition(c, tau, OK)


--- 元 x = 2*a^3 - 6*a^2 + 4*a - 5 の検証 ---
不変性チェック成功: σ(x) = 2*a^3 - 6*a^2 + 4*a - 5 で、x と一致します。
分解できる可能性があります。
ノルム: 1
ノルムは平方数です。分解できる可能性があります。


True

In [60]:
from itertools import product
R = range(3)
frag = False

u0, u1, u2 = U.gens_values()
for i,j,k in product(R,R,R):
    for e, d, f in product([1,-1],[1,-1],[1,-1]):
        u = (u0**(i*e))*(u1**(j*d))*(u2**(k*f))
        frag = u*tau(u) == c
        if frag:
            print((i*e,j*d,k*f), u)
            break
    if frag: break                    

(0, -1, -2) a^3 - 2*a^2 + 2*a - 2


In [44]:
u = (u0**0)*(u1**(-1))*(u2**(-2))
c, u, u*tau(u) == c 

(2*a^3 - 6*a^2 + 4*a - 5, a^3 - 2*a^2 + 2*a - 2, True)

### Scratch

In [ ]:
T = matrix([[-1,0,-2,-2],[0,1,2,0],[0,0,1,2],[0,0,0,1]])
display(T)
is_Sp(T)

In [ ]:
v = vector([a^3-2*a^2+2*a-2, a^2+1, -a, 1])
w = vector([a^2*(2-a), (a-1)^2, -(a-2), 1])
display(v,w, T*v, T*v == w)

In [ ]:
display(N*w, a*w)

In [ ]:
#display(X.inverse()*M*X)

In [ ]:
(a-2)*(1/a-2)*(-a^3+1/a^3), (a^3-1/a^3)-4*(a^2-1/a^2)+6*(a-1/a)

In [ ]:
#b = var('b')
#Delta = matrix([[1,a,a^2,a^3],[1,1/a,1/(a^2),1/(a^3)],[1,b,b^2,b^3],[1,1/b,1/(b^2),1/(b^3)]]).det()
Delta = matrix([[1,a],[1,1/a]]).det()
display(Delta)